In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import kelp_param
import kelp3d_objs as k3
import ipyparallel as ipp
import concurrent.futures as cf
import itertools as it
#from sympy import divisors
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as opt
from scipy.interpolate import interp1d
import ipyvolume as ipv
import ipywidgets as ipw

import kelp_compute
import kelp_analyze

In [3]:
import sqlite3
import shutil
import os

### Define Constants

In [4]:
out = ipw.Output()

In [5]:
out

Output()

In [6]:
# Hard-coded parameters
rope_spacing = 20.0 # (m)
zmin, zmax = 0.0, 10.0 # (m)

# Light from above
I0 = 50.0 # (W/m^2)
phi_s = 0 # (rad)
theta_s = 0 # (rad)
decay = 0 # (?) - light (practically) only from directly above

# Calculated or otherwise boring parameters
xmin = -rope_spacing/2
xmax = rope_spacing/2
ymin = -rope_spacing/2
ymax = rope_spacing/2

# Set parameters
study_name = 'single'
a_water = 0.179
b = 0.219
kelp_dist='top-heavy'

base_dir = '../../results'

ns = 20
nz = 20
na = 8

num_scatters = 1
fd_flag = False
lis_options = "-i gmres -restart 10000 -tol 1e-4"

# Compute
study_dir = os.path.join(base_dir, study_name)
data_dir = os.path.join(study_dir, 'data')

# TODO: Don't remove results?
shutil.rmtree(study_dir, ignore_errors=True)

os.makedirs(data_dir)
db_path = os.path.join(study_dir, '{}.db'.format(study_name))
conn = sqlite3.connect(db_path)
kelp_compute.create_table(conn, study_name)
conn.close()

with out:
    kelp_compute.kelp_calculate(
        a_water, b,
        ns, nz, na,
        kelp_dist, num_scatters,
        fd_flag,
        lis_options,
        study_dir=study_dir,
        study_name=study_name
    )

In [7]:
# Retrieve

conn = sqlite3.connect(db_path)
res = kelp_analyze.query_results(conn, study_name, ns=ns, nz=nz, na=na)[0]
conn.close()
irrad = res['irrad'][:]

print("{:.2f}% nan".format(np.sum(np.isnan(irrad))/np.size(irrad)))

# Visualize

ipv.figure()
ipv.volshow(irrad)
ipv.show()

IndexError: list index out of range